In [16]:
!pip install python-dotenv pandas altair seaborn requests

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.4.1-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (35 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached charset_normalizer-3.4.1-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (144 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.3.0-py3-none-any.whl (128 kB)


In [109]:
import pandas as pd
import requests
import json

import pathlib
import matplotlib as plt
import seaborn
import altair as alt

from os import getenv
from dotenv import load_dotenv

load_dotenv()
api_key = getenv("API_KEY_ALIGULAC")

# Goals
Get an understanding of variability of matchups within each race.

Dive into top 25 players and look at winrate to matchups and uncertainty associated with them.

# Initial API call to get top players and IDs

In [125]:
# Set the search parameters
target = "activerating"

params = {
    # "period": 95,
    "period" : 365,
    "order_by" : "-rating",
    "limit" : 1000,
    "format": "json",
    "apikey": api_key
}

base_url = f"http://aligulac.com/api/v1/{target}/?"

In [114]:
response = requests.get(base_url, params=params).json()
print(json.dumps(response, indent=4, sort_keys=True))


{
    "meta": {
        "limit": 1000,
        "next": null,
        "offset": 0,
        "previous": null,
        "total_count": 661
    },
    "objects": [
        {
            "bf_dev": 0.066315314649712,
            "bf_dev_vp": 0.0697187645699046,
            "bf_dev_vt": 0.0622031617494053,
            "bf_dev_vz": 0.0643622023513226,
            "bf_rating": 2.82528071280529,
            "bf_rating_vp": 0.0956339713869921,
            "bf_rating_vt": 0.180023550781798,
            "bf_rating_vz": -0.27565752216879,
            "comp_rat": -1000.0,
            "comp_rat_vp": -2000.0,
            "comp_rat_vt": -2000.0,
            "comp_rat_vz": 2.77542486646628,
            "decay": 0,
            "dev": 0.0885793296874642,
            "dev_vp": 0.0955817672171672,
            "dev_vt": 0.0805300501309756,
            "dev_vz": 0.0846635579328364,
            "domination": 0.505484321775544,
            "id": 9277809,
            "period": "/api/v1/period/365/",
            "p

In [116]:
objs = response['objects']
top_many = [(item['player']['tag'],
           item['player']['race'],
           item['position'],
           item['position_vp'],
           item['position_vt'],
           item['position_vz'],
           item['player']['country'],
           item['player']['id']) for item in objs]
df_top_many = pd.DataFrame(top_many, columns=['Name', 'Race', 'Rank', 'Rank_vP', 'Rank_vT', 'Rank_vZ', 'Country', 'Player_id'])
df_top_many.tail(3)

,Name,Race,Rank,Rank_vP,Rank_vT,Rank_vZ,Country,Player_id
658,Davelitoss,Z,659,660,659,658,SE,22037
659,GFG,T,660,659,660,660,DE,20099
660,imius,Z,661,661,661,661,RU,17287


# Filter for only top 500 players

In [148]:
df_top_500 = df_top_many[df_top_many['Rank'] <= 500]
df_top_500.tail(3)

,Name,Race,Rank,Rank_vP,Rank_vT,Rank_vZ,Country,Player_id
497,Senna,T,498,488,486,504,US,24627
498,RaThak,P,499,477,502,498,CZ,25329
499,Gamgam,P,500,487,490,503,SK,24637


# Grab IDs of top 25 players of each matchup for each race 

In [151]:
top_P = df_top_500[df_top_500['Race'].eq("P")]
top_PvP = top_P.sort_values('Rank_vP', ascending=True)
top_PvP.insert(1, 'Rank_PvP', range(1, 1 + len(top_PvP)))

In [153]:
top_PvPT = top_PvP.sort_values('Rank_vT', ascending=True)
top_PvPT.insert(1, 'Rank_PvT', range(1, 1 + len(top_PvPT)))

In [154]:
top_PvPTZ = top_PvPT.sort_values('Rank_vZ', ascending=True)
top_PvPTZ.insert(1, 'Rank_PvZ', range(1, 1 + len(top_PvPT)))

In [155]:
top_PvPTZ

,Name,Rank_PvZ,Rank_PvT,Rank_PvP,Race,Rank,Rank_vP,Rank_vT,Rank_vZ,Country,Player_id
5,herO,1,2,2,P,6,7,6,2,KR,233
2,MaxPax,2,1,1,P,3,4,3,5,DK,19591
16,ShoWTimE,3,5,5,P,17,22,19,12,DE,2170
14,Classic,4,4,3,P,15,19,16,13,KR,186
19,Creator,5,6,6,P,20,24,20,14,KR,2
...,...,...,...,...,...,...,...,...,...,...,...
487,LegolandM,180,170,171,P,488,473,471,493,IL,23550
498,RaThak,181,176,173,P,499,477,502,498,CZ,25329
499,Gamgam,182,173,177,P,500,487,490,503,SK,24637
496,Soil,183,166,178,P,497,492,461,520,CA,23332
